In [1]:
import pandas as pd
import numpy as np
from function_wrappers import *
from functions import *
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from functions import hyperparameter_grid

## Trading Linear Regression Best Window

###### Assignment

Take a window of W days.  Train a linear model on "Adj Close" prices with W days lagged.  Determine if the next day will be higher or lower than today.  Enter or stay in a position that is either long, short, or neutral depending on whether the price is expected to be higher, lower or the same.  Optimize for W according to returns in year 2.


###### Questions


1. take W = 5, 6, . . . , 30 and consider your data for year 1.
For each W in the specified range, compute your average
P/L per trade and plot it: on x-axis you plot the values
of W and on the y axis you plot profit and loss per trade.
What is the optimal value W∗
of W?
2. use the value of W∗
from year 1 and consider year 2. For
every day in year 2, take the previous W∗ days, compute
linear regression and compute the value of r
2
for that day.
Plot the graph of r
2
for year 2. What is the average r
2
.
How well does it explain price movements?
3. take the optimal value of W∗
from year 1 and use it to
implement the above trading strategy for year 2. How many
”long position” and ’short position” transactions did you
have in year 2?
4. what is the average profit/loss per ”long position” trade
and per ”short position” trades in year 2?
5. what is the average number of days for long position and
short position transactions in year 2?
6. are these results very different from those in year 1 for this
value of W∗
?

In [2]:
gme = pd.read_csv('./gme.csv')
gme.drop(['Date', 'Month', 'Day','Week_Number','High', 'Low', 'Close', 'Volume', 
         'Short_MA', 'Long_MA'], inplace=True, axis=1)

In [3]:
gme[gme['Year']==2020].index[0]

754

In [4]:
X_transform(gme,y=2020,w=5,d=1,measure='Adj Close')

array([[1.  , 1.36, 1.35, 1.48, 1.53, 1.52],
       [1.  , 1.35, 1.48, 1.53, 1.52, 1.58],
       [1.  , 1.48, 1.53, 1.52, 1.58, 1.47],
       ...,
       [1.  , 3.88, 4.86, 5.14, 5.04, 5.25],
       [1.  , 4.86, 5.14, 5.04, 5.25, 4.84],
       [1.  , 5.14, 5.04, 5.25, 4.84, 4.82]])

In [5]:
from functions import polynomial_accuracy

In [6]:
results = polynomial_accuracy(gme,y=2020,w=5,d=1,test=0,
                                  measure='Adj Close',weekly=False)
predicted_labels = results[-1]['predicted_labels']

In [7]:
np.unique(results[-1]['true_labels'],return_counts=True)

(array([0, 1]), array([131, 122]))

In [8]:
np.unique(results[-1]['predicted_labels'], return_counts=True)

(array([0, 1]), array([ 55, 198]))

My hypothesis for why there would be more predicted up movements is that there the up movements are on average higher than down movements, and therefore rmse is reduced by biasing predictions upwards.

In [9]:
predicted_labels

array([1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0,
       1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1,
       0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1])

Note: in the presence of short-selling, since one can lose more than principal, it makes sense to think of return as being on a scale from negative infinity to infinity, where positive returns indicate the proportion of the principal that is gained and negative returns indicate the proportion of the pricipal that is lost.  With only long positions, a range of (0,positive_infinity) to indicate the proportion of principal at the end of an investment period made sense.  For compatibility between long and short positions, I will proceed with the standard of negative_infinity, positive_infinity.

In [10]:
class position():
    def __init__(self, entry_price, position=1, position_size=100):
        self.position = position; self.position_size = position_size; self.entry_price = entry_price
        self.current_price = current_price; self.shares = position_size/entry_price; self.cumulative_return = 1
        self.profit = 0; self.log =[]
    def __str__(self):
        return f'position:{position}, entry price:{entry_price}, \
        current price:{current_price}'
    def update(current_price):
        self.current_price = current_price
        self.cumulative_return = (current_price - self.entry_price)*(-1)**(-1+2*position)/self.entry_price
        self.profit = self.position_size*(self.cumulative_return)

idea: dictionary of matrices

In [29]:
class trading_booth():
    """for caclulating returns with a given window for prices and labels"""
    def __init__(self, stock, window = 5, training_year = 2020, evaluation_year = 2021):
        self.stock = stock
        # self.prices = prices; self.array_labels = array_labels; self.window = window; 
        self.training_year = training_year;
        self.evaluation_year = evaluation_year;
        self.total_profit = 0; 
        self.log = {'long_trades':[], 'short_trades':[]}; 
        self.cumulative_return = 1
        stock = self.stock
        stock['begin_prices'] = [stock['Open'].values[0]]+\
                                list(stock['Adj Close'].values[1:])
        self.stock = stock
    def __str__(self):
        return f'profit: {self.total_profit}'
    def predict():
        pass
    # def create_X(self.window, self.trading_year,):
    def X_matrix(prices = self., w = 5):
        """Return a matrix of weekly lagged values for a given feature."""
        (begin,stop) = (prices.index[0],
                        stock[stock['Year']==y].index[-1]+1)
        X_lags = [stock[f'{measure}'][i-w:i].values for i in range(begin,stop)]
        return X_lags      
    def add_intercept(X_pe):
        """Return matrix with intercept prepended to each row."""
        X_pe = np.asarray(X_pe, dtype = 'float64')
        X_i_pe = np.vstack([np.ones(len(X_pe)),X_pe.T]).T
        return X_i_pe


SyntaxError: invalid syntax (3946853030.py, line 20)

In [30]:
gme_trading_booth = trading_booth(gme, window=5,training_year=2020,evaluation_year=2021)

In [31]:
gme_trading_booth.stock

,Year,Weekday,Year_Week,Open,Adj Close,Return
0,2017,Tuesday,2017-01,6.36,5.16,0.000000
1,2017,Wednesday,2017-01,6.39,5.27,0.021679
2,2017,Thursday,2017-01,6.39,5.08,-0.035494
3,2017,Friday,2017-01,6.29,4.99,-0.019200
4,2017,Monday,2017-02,6.15,4.98,-0.001224
...,...,...,...,...,...,...
1253,2021,Thursday,2021-51,38.50,38.03,-0.012078
1254,2021,Monday,2021-52,38.00,37.08,-0.025174
1255,2021,Tuesday,2021-52,36.88,36.62,-0.012474
1256,2021,Wednesday,2021-52,36.96,38.48,0.051004


In [ ]:
gme_trading_booth.X_matrix

In [13]:
gme

,Year,Weekday,Year_Week,Open,Adj Close,Return
0,2017,Tuesday,2017-01,6.36,5.16,0.000000
1,2017,Wednesday,2017-01,6.39,5.27,0.021679
2,2017,Thursday,2017-01,6.39,5.08,-0.035494
3,2017,Friday,2017-01,6.29,4.99,-0.019200
4,2017,Monday,2017-02,6.15,4.98,-0.001224
...,...,...,...,...,...,...
1253,2021,Thursday,2021-51,38.50,38.03,-0.012078
1254,2021,Monday,2021-52,38.00,37.08,-0.025174
1255,2021,Tuesday,2021-52,36.88,36.62,-0.012474
1256,2021,Wednesday,2021-52,36.96,38.48,0.051004


In [14]:
gme_trading_booth.stock

,Year,Weekday,Year_Week,Open,Adj Close,Return
0,2017,Tuesday,2017-01,6.36,5.16,0.000000
1,2017,Wednesday,2017-01,6.39,5.27,0.021679
2,2017,Thursday,2017-01,6.39,5.08,-0.035494
3,2017,Friday,2017-01,6.29,4.99,-0.019200
4,2017,Monday,2017-02,6.15,4.98,-0.001224
...,...,...,...,...,...,...
1253,2021,Thursday,2021-51,38.50,38.03,-0.012078
1254,2021,Monday,2021-52,38.00,37.08,-0.025174
1255,2021,Tuesday,2021-52,36.88,36.62,-0.012474
1256,2021,Wednesday,2021-52,36.96,38.48,0.051004


In [15]:
dir(gme_trading_booth)

['X_matrix',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'add_intercept',
 'cumulative_return',
 'evaluation_year',
 'log',
 'predict',
 'stock',
 'total_profit',
 'training_year']

In [16]:
gme_trading_booth.stock

,Year,Weekday,Year_Week,Open,Adj Close,Return
0,2017,Tuesday,2017-01,6.36,5.16,0.000000
1,2017,Wednesday,2017-01,6.39,5.27,0.021679
2,2017,Thursday,2017-01,6.39,5.08,-0.035494
3,2017,Friday,2017-01,6.29,4.99,-0.019200
4,2017,Monday,2017-02,6.15,4.98,-0.001224
...,...,...,...,...,...,...
1253,2021,Thursday,2021-51,38.50,38.03,-0.012078
1254,2021,Monday,2021-52,38.00,37.08,-0.025174
1255,2021,Tuesday,2021-52,36.88,36.62,-0.012474
1256,2021,Wednesday,2021-52,36.96,38.48,0.051004


In [17]:
cumulative_return = 0
standing_trade = predicted_labels[0]
standing_trade
for p in predicted_labels[1:]:
    if p == 

SyntaxError: invalid syntax (1811510385.py, line 5)

In [ ]:
predictions = results[-1]['predictions']

In [ ]:
for p in predictions:

In [ ]:
gme

In [ ]:
gme.query('Year == 2020')

In [ ]:
predictions

Long trade implementation:
Short trade implementation:

In [ ]:
begin_prices = [gme.loc[0,'Open']]
begin_prices = begin_prices+list(gme.loc[:gme.index[-2],'Adj Close'].values)

In [ ]:
len(begin_prices)

In [ ]:
gme['Begin'] = begin_prices

In [ ]:
gme['Begin']

In [ ]:
gme['ReturnCalc'] = gme['Adj Close']/gme['Begin']